In [119]:
import sys
import numpy as np
from matplotlib import pyplot as plt
import h5py
import pandas as pd
from collections import OrderedDict

%matplotlib notebook

In [2]:
import qcodes as qc

# Setup notes

For this example to run, you need to have set up a labpython config file. There is a template in labpythonQDelft/templates.

In [3]:
from importlib import reload
from qctools import instruments as instools
reload(instools)

import labpythonconfig as cfg
reload(cfg)

from experiment import measurement
reload(measurement)
from experiment.measurement import Parameter, BaseMeasurement

In [4]:
from pysweep.data_storage import np_storage
reload(np_storage)
from pysweep.data_storage.np_storage import NpStorage, DataSet

In [5]:
station = qc.Station()

In [162]:
import os
import h5py
import numpy as np

class Data(NpStorage):
    def __init__(self, filepath):
        super().__init__()
        self.filepath = filepath
        
    
    def spyview_meta(self, page):
        pass       
            
    @staticmethod
    def flatten_page(page):
        names = [n for n in page.dtype.fields]
        shapes = [page.dtype[n].shape for n in names]
        dtypes = [page[n].dtype for n in names]
        sizes = [np.prod(s) for s in shapes]

        if ((1,) in shapes and len(set(shapes)) > 2) or ((1,) not in shapes and len(set(shapes)) > 1):
            raise NotImplementedError("Flattening of non-trivial shapes is currently not implemented.")
        
        blocksize = max(sizes)
        pagelen = page.size
        datasize = blocksize * pagelen
        data = np.zeros((datasize,), dtype=[(n,d) for n,d in zip(names, dtypes)])
        
        for name, size in zip(names, sizes):
            if size < blocksize:
                data[name] = np.vstack(blocksize*[page[name].reshape(-1)]).reshape(-1, order='F')
            else:
                data[name] = page[name].reshape(-1)
        
        return data

    def write_spyview_data(self, fn, page):
        isnewfile = not os.path.exists(fn)
        
        data = Data.flatten_page(page)
        _names = [n for n in data.dtype.fields]
        names = _names[::-1][1:] + [_names[-1]]
        page_name = names[-1]
        
        alldata = Data.flatten_page(self[page_name])
        col0_start = alldata[names[0]][0]
      
        with open(fn, 'a') as f:
            for i, row in enumerate(data):
                if row[names[0]] == col0_start:
                    f.write("\n")
                
                line = "\t".join(["{}".format(row[k]) for k in names])
                line += "\n"
                f.write(line)
        
    
    def add(self, record, write=True):
        super().add(record)
        if write:
            self.write(record)    
        
    def write(self, record=None):
        with h5py.File(self.filepath, 'a') as f:
            if record is None:
                pages = self._pages
            else:
                pages = self.record_to_pages(record)

            for name, arr in pages.items():
                if name in f:
                    d = f[name]
                else:
                    d = f.create_dataset(name, (0,), maxshape=(None,), dtype=arr.dtype, chunks=True)
                    
                newlen = d.size + arr.size
                d.resize((newlen,))
                d[-arr.size:] = arr

In [163]:
data = Data(None)
        
# set up loop
xpoints = np.linspace(0, 10, 3)
ypoints = np.linspace(0, 10, 5)

# now populate in a primitive loop
for x in xpoints:
    for y in ypoints:

        # acquire some data
        z = np.cos(x) * np.cos(y) * np.linspace(0,1,4).reshape(2,2)
        idx = np.arange(4).reshape(2,2)

        # add to dataset
        data.add(
            OrderedDict(
                {'a' : {'value' : x, 'independent_parameter' : True, 'unit' : 'V'},
                 'b' : {'value' : y, 'independent_parameter' : True, 'unit' : 'V'},
                 'idx' : {'value' : idx, 'independent_parameter' : True},
                 'z' : {'value' : z },
            }), write=False
        )

In [164]:
data.write_spyview_data('test.txt', data['z'])